# get deta on all incidents of force by police dept

In [1]:
import pandas as pd
from selenium import webdriver
from pathlib import Path
from datetime import datetime as dt 
today = dt.today().strftime("%Y-%m-%d")

# THIS_FILE = Path(__file__)
# THIS_DIR = THIS_FILE.parent
# CHROMEDRIVER_PATH = THIS_DIR.joinpath("./../../notebooks/chromedriver")
# INTERIM_DATA = THIS_DIR.joinpath("./../../data/interim/")
# PROCESSED_DATA = THIS_DIR.joinpath("./../../data/processed/")

BASE_URL = "http://force.nj.com"

In [2]:
browser = webdriver.Chrome("./chromedriver")
# Compile list police dept of options from dropdown menu
browser.get(BASE_URL)

police_depts_list = browser.find_element_by_id("mylist")
police_depts = police_depts_list.find_elements_by_tag_name("option")

In [3]:
depts_list = []
for dept in police_depts:
    dept_info = {}
    dept_info['name'] = dept.get_attribute("innerHTML").strip()
    dept_info['relative_url'] = dept.get_attribute("value")
    depts_list.append(dept_info)

In [4]:
browser.get(BASE_URL + depts_list[0]['relative_url'])

In [5]:
def get_incidents_table():
    """Retrieve the incidents table presented by default in each police depts page.
        
    Returns
    -------
    table : pd.DataFrame
       table containing `n_rows` officers per dept
    """
    incidents_table = browser.find_element_by_id("incidents_table").get_property("outerHTML")
    table = pd.read_html(incidents_table)[0]
    voi = [col for col in table if 'Unnamed' not in col]
    return table[voi]

In [36]:
def get_number_of_total_rows():
    sentence = browser.find_element_by_id("incidents_table_info").get_property("innerHTML")
    n_rows = int(sentence.split(" of ")[-1].split()[0])
    return n_rows

In [16]:
def change_table_length():
    browser.find_element_by_xpath("//select[@name='incidents_table_length']/option[text()='100']").click()

In [17]:
change_table_length()

In [12]:
total_n_rows = int(get_number_of_total_rows().split(" of ")[-1].split()[0])

In [22]:
total_n_rows // 100

1

1. change to 100
2. figure out how many times to click next
3. grab table
4. click next, 
5. repeat 3 and 4 as needed
6. concatenate tables
7. save tables to name of county

In [29]:
test_depts = depts_list[:5]

In [38]:
for dept in test_depts:
    browser.get(BASE_URL + dept['relative_url'])
    try:
        print("changing table length...")
        change_table_length()
        total_n_rows = get_number_of_total_rows()
        n_rounds = (total_n_rows // 100) + 1
        print(f"Getting table for {dept['name']}: {total_n_rows} entries.")
        tables = []
        for page in range(n_rounds):
            print(f"round {page}/{n_rounds}")
            tables.append(get_incidents_table())
            browser.find_element_by_id("incidents_table_next").click()
        df = pd.concat(tables)
        dept['tables'] = df
    except:
        pass

changing table length...
Getting table for Aberdeen, Monmouth: 151 entries.
round 0
round 1
changing table length...
Getting table for Absecon, Atlantic: 58 entries.
round 0
changing table length...
Getting table for Allendale, Bergen: 10 entries.
round 0
changing table length...
Getting table for Allenhurst, Monmouth: 9 entries.
round 0
changing table length...
Getting table for Allentown, Monmouth: 7 entries.
round 0


In [39]:
test_depts

[{'name': 'Aberdeen, Monmouth',
  'relative_url': '/database/pd-dept/aberdeen-monmouth',
  'tables':            Officer name      Date   Time                 Type of force  \
  0         Michael Lasko   12/6/16  22:02               compliance hold   
  1         Michael Plant  10/21/16  16:05  compliance hold, hands/fists   
  2   Michael Rzigalinski   10/6/16  11:01               compliance hold   
  3            Gus Grivas   10/6/16  11:00               compliance hold   
  4      Raymond Campbell   10/3/16  18:37               compliance hold   
  ..                  ...       ...    ...                           ...   
  46        Matthew Lloyd    3/2/12  16:43               compliance hold   
  47       Darrell Oliver    1/2/12   1:08               compliance hold   
  48       Craig Hausmann    1/2/12   3:21               compliance hold   
  49     Jeffrey Griffith    1/1/12  20:49  compliance hold, hands/fists   
  50          Louis Nanna    1/1/12  23:30  compliance hold, hand

In [28]:
browser.find_element_by_id("incidents_table_next").click()

In [18]:
get_incidents_table()

,Officer name,Date,Time,Type of force,Reason,Subject age
0,Raymond Sosa,10/1/13,15:50,compliance hold,"resisted police officer control, suicide attempt",56
1,Martin Walsh,9/28/13,5:41,compliance hold,resisted police officer control,47
2,Matthew McDow,9/19/13,18:31,compliance hold,resisted police officer control,18
3,John Young,9/10/13,17:06,compliance hold,attempted take off firearm,38
4,Edgar Serrano,8/10/13,0:46,compliance hold,resisted police officer control,22
5,Frank Maisano,7/3/13,18:20,compliance hold,"resisted police officer control, spit in offic...",51
6,Mary Johnston,6/29/13,17:11,"compliance hold, hands/fists","resisted police officer control, physical thre...",30
7,Frank Sigismondi,6/29/13,18:13,"compliance hold, hands/fists","resisted police officer control, physical thre...",30
8,John Young,6/29/13,17:22,"compliance hold, hands/fists","resisted police officer control, physical thre...",30
9,Jeffrey Griffith,6/20/13,23:12,"compliance hold, hands/fists",resisted police officer control,41


In [28]:
df = pd.read_html(table)[0]

In [29]:
df

,Unnamed: 0,Officer name,Date,Time,Type of force,Reason,Subject age,Subject sex,Subject race,Officer race
0,NaN,Michael Lasko,12/6/16,22:02,compliance hold,"resisted police officer control, physical thre...",55,male,white,white
1,NaN,Michael Plant,10/21/16,16:05,"compliance hold, hands/fists","resisted police officer control, physical thre...",38,female,white,white
2,NaN,Michael Rzigalinski,10/6/16,11:01,compliance hold,"resisted police officer control, physical thre...",43,male,white,white
3,NaN,Gus Grivas,10/6/16,11:00,compliance hold,"resisted police officer control, physical thre...",43,male,white,white
4,NaN,Raymond Campbell,10/3/16,18:37,compliance hold,resisted police officer control,18,male,black,white
5,NaN,John Young,10/3/16,18:35,compliance hold,resisted police officer control,18,male,black,white
6,NaN,David Mauro,10/3/16,18:38,compliance hold,resisted police officer control,18,male,black,white
7,NaN,Gus Grivas,7/20/16,11:23,compliance hold,resisted police officer control,58,female,white,white
8,NaN,Henry Chevalier,7/20/16,11:16,compliance hold,resisted police officer control,58,female,white,white
9,NaN,Christopher Desarno,6/17/16,12:08,"compliance hold, hands/fists","resisted police officer control, physical thre...",57,female,white,white


In [9]:
pd_info = browser.find_element_by_class_name("pd_info")
town_info = pd_info.find_element_by_class_name("town_description")
town_info.find_element_by_class_name("right").find_element_by_class_name("town_label").get_property("innerHTML")

'Average number of full-time officers: 34'

In [6]:
for dept in depts_list[:15]:
    # go to page
    browser.get(BASE_URL + dept['relative_url'])

    print(f"Getting info for {dept['name']}")
    
    # get likelihood numbers
    # by population
    try:
        (prop, number) = get_likelihood_numbers('left')
        dept[prop] = number
    except:
        dept['population'] = 'Not found'
    # by arrests
    try:
        (prop, number) = get_likelihood_numbers('right')
        dept[prop] = number
    except:
        dept['arrests'] = 'Not found'
    
    # get no of officers that would be flagged in other cities
    try:
        (first_city, first_city_n, second_city, second_city_n, third_city, third_city_n) = get_flagging_officer_numbers()
        dept[first_city] = first_city_n
        dept[second_city] = second_city_n
        dept[third_city] = third_city_n
    except:
        dept['Los Angeles'] = 'Not found'
        dept['New York City'] = 'Not found'
        dept['Chicago'] = 'Not found'

browser.close()

Getting info for Aberdeen, Monmouth
Getting info for Absecon, Atlantic
Getting info for Allendale, Bergen
Getting info for Allenhurst, Monmouth
Getting info for Allentown, Monmouth
Getting info for Alpine, Bergen
Getting info for Andover Township, Sussex
Getting info for Asbury Park, Monmouth
Getting info for Atlantic City, Atlantic
Getting info for Atlantic Highlands, Monmouth
Getting info for Audubon, Camden
Getting info for Avalon, Cape May
Getting info for Avon-By-the-Sea, Monmouth
Getting info for Barnegat, Ocean
Getting info for Barrington, Camden


In [7]:
data = pd.DataFrame(depts_list)
data['full_url'] = BASE_URL + data['relative_url']

In [8]:
data

,name,relative_url,population,arrests,Los Angeles,New York City,Chicago,full_url
0,"Aberdeen, Monmouth",/database/pd-dept/aberdeen-monmouth,294%,42%,Not found,Not found,Not found,http://force.nj.com/database/pd-dept/aberdeen-...
1,"Absecon, Atlantic",/database/pd-dept/absecon-atlantic,583%,68%,0,9,13,http://force.nj.com/database/pd-dept/absecon-a...
2,"Allendale, Bergen",/database/pd-dept/allendale-bergen,N/A,8%,0,2,3,http://force.nj.com/database/pd-dept/allendale...
3,"Allenhurst, Monmouth",/database/pd-dept/allenhurst-monmouth,N/A,N/A,0,4,4,http://force.nj.com/database/pd-dept/allenhurs...
4,"Allentown, Monmouth",/database/pd-dept/allentown-monmouth,N/A,N/A,0,1,1,http://force.nj.com/database/pd-dept/allentown...
...,...,...,...,...,...,...,...,...
542,"Knowlton Township, Warren",/database/pd-dept/knowlton-township-warren,NaN,NaN,NaN,NaN,NaN,http://force.nj.com/database/pd-dept/knowlton-...
543,"Liberty Township, Warren",/database/pd-dept/liberty-township-warren,NaN,NaN,NaN,NaN,NaN,http://force.nj.com/database/pd-dept/liberty-t...
544,"Oxford, Warren",/database/pd-dept/oxford-warren,NaN,NaN,NaN,NaN,NaN,http://force.nj.com/database/pd-dept/oxford-wa...
545,"White, Warren",/database/pd-dept/white-warren,NaN,NaN,NaN,NaN,NaN,http://force.nj.com/database/pd-dept/white-warren
